# Present the results

In [ ]:
import pandas as pd
import yfinance as yf
import tensorflow as tf 
from tensorflow.keras.models import load_model
import utils_datasets as utils_data
import utils_results as utils_result
import importlib
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [ ]:
TICKER = 'UPS'
#the model must be stored with a .keras file
path_model = ''
#use the path of the datasets that contains all the datas
path_images = ''

"""--------------------------------RISK FREE RATE------------------------------------"""

RISK_FREE_DF =  yf.download("^IRX", start="2000-01-01")
RISK_FREE_DF.columns = RISK_FREE_DF.columns.get_level_values(0)
RISK_FREE_DF = RISK_FREE_DF[['Open', 'Close']].copy()
RISK_FREE_DF.columns.name = None 

"""--------------------------------DIVIDEND------------------------------------------"""

stock_ticker = yf.Ticker(TICKER)
DIVIDEND = stock_ticker.info.get('dividendYield', 0.0)


"""---------------------------------DATA STOCK---------------------------------------"""

TICKER_DATA_DATE = yf.download(TICKER, start="2000-01-01", auto_adjust=True, interval='1d')
TICKER_DATA_DATE.columns = TICKER_DATA_DATE.columns.get_level_values(0)
TICKER_DATA_DATE = TICKER_DATA_DATE[['Open', 'Close', 'High', 'Low', 'Volume']].copy()
TICKER_DATA_DATE.columns.name = None 
TICKER_DATA_DATE['MA20'] = TICKER_DATA_DATE['Close'].rolling(window=20).mean()

"""---------------------------------MODEL--------------------------------------------"""
MODEL = load_model(path_model)

"""---------------------------------DATASET PICTURES---------------------------------"""

DF_KERAS = utils_data.load_all_ticker_data(path_images)
DF_KERAS = DF_KERAS[DF_KERAS['ticker'] == TICKER]


# GET WINDOW
To get the corresponding window, use the version of the dataset where no file have been deleted.s

In [ ]:
df_ticker = utils_data.download_historical_prices(TICKER)
df_ticker['event'] = df_ticker['Close'] > 1.007 * df_ticker['Open']
df_ticker['event_count'] = df_ticker['event'].rolling(window=40).sum()
max_idx = df_ticker['event_count'].idxmax()
START_IDX = max_idx - 39
END_IDX = max_idx
print(START_IDX, END_IDX)


# PLOT

In [ ]:
importlib.reload(utils_result)
#END_IDX = END_IDX+250
fig = utils_result.get_plot(TICKER_DATA_DATE, START_IDX, END_IDX)

# GET OPTION PRICE

In [ ]:
importlib.reload(utils_result)
STRIKE = 32
MATURITY = '2009-10-10'
DF_OPTIONS = pd.DataFrame()
for i, rows in TICKER_DATA_DATE.loc['2008-10-10':'2009-10-10'].iterrows():
    call_row = {}
    date = rows.name
    spot = rows['Close']
    vol = utils_result.get_realized_volatility(TICKER_DATA_DATE, date)
    #print(f'Pricing date : {date}')
    rf = RISK_FREE_DF.loc[date,'Close']/100
    price_call, days_remaining = utils_result.black_scholes_call_price(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    delta = utils_result.call_delta(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    gamma = utils_result.call_gamma(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    vega = utils_result.call_vega(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    date = date.strftime('%Y-%m-%d')
    call_row = {
        'Pricing date': date,
        'Call_price': price_call,
        'Delta': delta,
        'Gamma': gamma,
        'Vega': vega,
        'Days_remaining': days_remaining,
        'Spot': spot
    }
    DF_OPTIONS = pd.concat([DF_OPTIONS, pd.DataFrame([call_row])], axis=0)
    
DF_OPTIONS = DF_OPTIONS.set_index('Pricing date')

DF_OPTIONS_OPEN = pd.DataFrame()
for i, rows in TICKER_DATA_DATE.loc['2008-10-10':'2009-10-10'].iterrows():
    call_row = {}
    date = rows.name
    spot = rows['Open']
    vol = utils_result.get_realized_volatility(TICKER_DATA_DATE, date)
    #print(f'Pricing date : {date}')
    rf = RISK_FREE_DF.loc[date,'Open']/100
    price_call, days_remaining = utils_result.black_scholes_call_price(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    delta = utils_result.call_delta(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    gamma = utils_result.call_gamma(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    vega = utils_result.call_vega(date, MATURITY, spot, STRIKE,  rf, DIVIDEND/100, vol)
    date = date.strftime('%Y-%m-%d')
    call_row = {
        'Pricing date': date,
        'Call_price': price_call,
        'Delta': delta,
        'Gamma': gamma,
        'Vega': vega,
        'Days_remaining': days_remaining,
        'Spot': spot
    }
    DF_OPTIONS_OPEN = pd.concat([DF_OPTIONS_OPEN, pd.DataFrame([call_row])], axis=0)
    
DF_OPTIONS_OPEN = DF_OPTIONS_OPEN.set_index('Pricing date')
display(DF_OPTIONS_OPEN.head(5))
display(DF_OPTIONS.head(5))


In [ ]:
DF_OPTIONS

# Basic hedging
Imagine we set up a position of 1000 call:
- Strike 34
- Spot: 30

We calculate the cost of hedging throug the days

In [ ]:
importlib.reload(utils_result)
import plotly.graph_objects as go

notional = 10000
res = utils_result.get_hedgigng_dataframe(DF_OPTIONS, TICKER_DATA_DATE, notional)


import plotly.graph_objects as go

# Get the last column name
right_col = res.columns[-1]
left_cols = res.columns[:-1]

fig = go.Figure()

# Left y-axis: all except last column
for i, col in enumerate(left_cols):
    fig.add_trace(go.Scatter(
        x=res.index,
        y=res[col],
        mode='lines',
        name=col,
        yaxis='y1',
        visible=True if i == 0 else 'legendonly'
    ))

# Right y-axis: only the last column
fig.add_trace(go.Scatter(
    x=res.index,
    y=res[right_col],
    mode='lines',
    name=right_col,
    yaxis='y2',
    visible='legendonly'
))

# Layout
fig.update_layout(
    title="Delta Hedging: Share Activity, Prices & Hedging Cost",
    xaxis=dict(title="Date"),
    yaxis=dict(
        title="Main Metrics",
        side='left'
    ),
    yaxis2=dict(
        title=right_col,
        overlaying='y',
        side='right'
    ),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.12,
        xanchor="center",
        x=0.5
    ),
    template="plotly_dark",
    hovermode="x unified",
    margin=dict(t=120, b=60, l=80, r=80),
    width=1200,  # 👈 adjust width here
    height=600   # 👈 adjust height here
)

fig.show()






# Hedging with a model

In [ ]:
importlib.reload(utils_result)
first_day_hedging = res.iloc[0,:]
res_model = pd.DataFrame(first_day_hedging).T
res_model['Schedule'] = ['Close']
for i, row in DF_OPTIONS_OPEN.iloc[1:,].iterrows():
    delta = row['Delta']
    gamma = row['Gamma']
  

    price_open = TICKER_DATA_DATE.loc[i]['Open']
    price_close = TICKER_DATA_DATE.loc[i]['Close']

    index_number = TICKER_DATA_DATE.index.get_loc(i)
    index_pic = index_number - 39 + 1
    if index_number == 30:
         print(i)
    pic_tf = utils_data.dataframe_to_dataset_color(DF_KERAS[DF_KERAS['id'] == index_pic])
    X_batch, y_batch = next(iter(pic_tf))
    preds =  MODEL.predict(np.expand_dims(X_batch, axis=0), verbose=0)
    pred_class =  np.argmax(preds, axis=1) 
   


    # pre hedging:

    if pred_class == [1]:
        dict_day = {}
        index = row.name
        dict_day['date'] = index
        shadow_delta = delta + gamma * 0.015*price_open
        dict_day['delta'] = shadow_delta

        shadow_delta_shares = round(notional * shadow_delta,2)
        
        old_delta_shares = res_model.iloc[-1].to_frame().T['total_number'].values

        shadow_to_buy = shadow_delta_shares - old_delta_shares
        shadow_price = price_open * shadow_to_buy

        dict_day['number_of_shares_bought_that_day'] =  shadow_to_buy
        dict_day['number_of_shares_sold_that_day'] = 0
        dict_day['price_per_share_that_day'] = price_open

        old_mean_buy = res_model.iloc[-1].to_frame().T['mean_buy_value'].values
        old_mean_sell = res_model.iloc[-1].to_frame().T['mean_sell_value'].values

        new_mean_buy = (res_model.iloc[-1].to_frame().T['total_number_bought'].values * old_mean_buy + shadow_to_buy * price_open)/(shadow_to_buy + res_model.iloc[-1].to_frame().T['total_number_bought'].values)
        new_mean_sell = old_mean_sell

        dict_day['mean_buy_value'] = new_mean_buy
        dict_day['mean_sell_value'] = new_mean_sell

        total_number_bought = res_model.iloc[-1].to_frame().T['total_number_bought'].values + shadow_to_buy
        total_number_sold = res_model.iloc[-1].to_frame().T['total_number_sold'].values

        dict_day['total_number_bought'] = total_number_bought
        dict_day['total_number_sold'] = total_number_sold

        total_number = total_number_bought - total_number_sold

        

        dict_day['total_number'] = total_number

        total_buy_value = total_number_bought * new_mean_buy
        total_sell_value = total_number_sold * new_mean_sell

        dict_day['total_buy_value'] = total_buy_value
        dict_day['total_sell_value'] = total_sell_value

        total_value = -(total_sell_value - total_buy_value)
        dict_day['delta_hedging_cost'] = total_value
        dict_day['Schedule'] = 'Open'
    
        res_model = pd.concat([res_model, pd.DataFrame(dict_day)], axis = 0)

        row_option_close = DF_OPTIONS.loc[i]
        second_hedging = utils_result.normal_hedging(row_option_close, notional, TICKER_DATA_DATE, res_model.iloc[-1].to_frame().T)
        res_model = pd.concat([res_model, second_hedging], axis = 0)
        
    else:
            row_option_close = DF_OPTIONS.loc[i]
            res_day = utils_result.normal_hedging(row_option_close, notional, TICKER_DATA_DATE, res_model.iloc[-1].to_frame().T)
            res_model = pd.concat([res_model, res_day], axis = 0, ignore_index=False)


res_model = res_model.set_index('date')
res_model


    



In [ ]:
# Get the last column name
right_col = res_model.columns[-2]
left_cols = res_model.columns[:-2]

fig = go.Figure()

# Left y-axis: all except last column
for i, col in enumerate(left_cols):
    fig.add_trace(go.Scatter(
        x=res_model.index,
        y=res_model[col],
        mode='lines',
        name=col,
        yaxis='y1',
        visible=True if i == 0 else 'legendonly'
    ))

# Right y-axis: only the last column
fig.add_trace(go.Scatter(
    x=res_model.index,
    y=res_model[right_col],
    mode='lines',
    name=right_col,
    yaxis='y2',
    visible='legendonly'
))

# Layout
fig.update_layout(
    title="Delta Hedging: Share Activity, Prices & Hedging Cost",
    xaxis=dict(title="Date"),
    yaxis=dict(
        title="Main Metrics",
        side='left'
    ),
    yaxis2=dict(
        title=right_col,
        overlaying='y',
        side='right'
    ),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.12,
        xanchor="center",
        x=0.5
    ),
    template="plotly_dark",
    hovermode="x unified",
    margin=dict(t=120, b=60, l=80, r=80),
    width=1200,  # 👈 adjust width here
    height=600   # 👈 adjust height here
)

fig.show()

In [ ]:
res_model['open_occurrence'] = (res_model['Schedule'] == 'Open').cumsum()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=res.index,
        y=res['delta_hedging_cost'],
        mode='lines',
        name='Normal Delta Hedging',
        yaxis='y1',
        visible=True,

    ))

# Right y-axis: only the last column
fig.add_trace(go.Scatter(
    x=res_model.index,
    y=res_model['delta_hedging_cost'],
    mode='lines',
    name='Delta Hedging with model',
    yaxis='y1',
    visible=True
))

fig.add_trace(go.Scatter(
    x=res_model.index,
    y=res_model['open_occurrence'],
    mode='lines',
    name='Class 1 prediction',
    yaxis='y2',
    visible=True
))

# Layout
fig.update_layout(
    title="Delta Hedging: Standard Vs Model informed",
    xaxis=dict(title="Date"),
    yaxis=dict(
        title="Costs",
        side='left'
    ),
    yaxis2=dict(
        title='Model interventions',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.12,
        xanchor="center",
        x=0.5
    ),
    template="plotly_dark",
    hovermode="x unified",
    margin=dict(t=120, b=60, l=80, r=80),
    width=1200,  # 👈 adjust width here
    height=600   # 👈 adjust height here
)

fig.show()

In [ ]:
(res.iloc[-1]['delta_hedging_cost'] - res_model.iloc[-1]['delta_hedging_cost'])/res.iloc[-1]['delta_hedging_cost']